# Parser function

This function iterates throught a given folder containing demos and parses them into json and csv files. The output can be found under the *parsed-demos* folder.

## Collecting the demo files in a folder

This function expects one parameter:
*   *folderPath*: the path of the folder in which the demos are located.

In the given folder, the function collects the names of the demo files and the relative path to them from this folder (*demo-parser*) and returns them in an array.

In [19]:
import os

def get_demo_files(folderPath):
    # Demo list
    demo_list = []

    # Iterate directory and collecting the demo files
    for filename in os.listdir(folderPath):
        filePath = os.path.join(folderPath, filename)
        if '.' not in filename:
            continue
        extension = filename.rsplit('.', 1)[1]
        tournament = folderPath.rsplit('\\', 1)[1]

        # Checking files and counting rars 
        if os.path.isfile(filePath):
            if extension == 'dem':
                demo_list.append('../demo/' + tournament + '/' + filename)

    file = open(tournament + '.txt', 'a+')
    file.close()
    return tournament, demo_list

## Parsing demos
Function for parsing the demo files in  given folder. It expects one parameter:
*   *folderPath*: which contains the location of the folder containing the demo files.

This function uses the earlier defined *get_demo_files()* function to collect the name and location (relative path) of the demo files in the given folder, than iterates through them and calls the **DemoParser** from the *awpy* package. The returned dataframes are saved to *parsed-demos/csv* folder in .csv format, as well as json (the path for those is *parsed-demos/json*).

In [16]:
from awpy import DemoParser
import pandas as pd


def parse_demos_in_folder(folderPath):
    # Get the demo list and tournament folder name
    tournament, demo_list = get_demo_files(folderPath)

    # Create the datasets in which the data from the matches will be saved
    rounds = pd.DataFrame()
    kills = pd.DataFrame()
    damages = pd.DataFrame()
    grenades = pd.DataFrame()
    flashes = pd.DataFrame()
    weaponFires = pd.DataFrame()
    bombEvents = pd.DataFrame()
    frames = pd.DataFrame()
    playerFrames = pd.DataFrame()

    # Temporary list to save parsed demo files
    temp_parsedDemos = []

    # Iterate through the demos
    idx = 0
    for demo in demo_list:
        # Check wether the demo is alreay parsed
        with open(tournament + '.txt') as f:
            lines = f.read().splitlines()
            if demo in lines:
                continue

        # Parse demo
        match_id = str(demo).replace('../demo/', '').replace('/', '-')
        demo_parser = DemoParser(demofile=demo, demo_id=match_id, parse_rate=128, outpath='../parsed-demos/json')
        data = demo_parser.parse(return_type='df')
        data['frames']['matchID'] = data['matchID']
        data['frames']['mapName'] = data['mapName']

        # Append the parsed demo's data to padas tables
        rounds = pd.concat([rounds, data['rounds']])
        kills = pd.concat([kills, data['kills']])
        damages = pd.concat([damages, data['damages']])
        grenades = pd.concat([grenades, data['grenades']])
        flashes = pd.concat([flashes, data['flashes']])
        weaponFires = pd.concat([weaponFires, data['weaponFires']])
        bombEvents = pd.concat([bombEvents, data['bombEvents']])
        frames = pd.concat([frames, data['frames']])
        playerFrames = pd.concat([playerFrames, data['playerFrames']])

        # Increase index, write new demos to file, and if enough demos are parsed, write them to file
        idx = idx + 1
        temp_parsedDemos.append(demo)
        print(demo + " done")

        if (idx % 7 == 0) or (idx == len(demo_list)-2) or (idx == len(demo_list) - 1):
            rounds.to_csv('../parsed-demos/csv/' + tournament + '_rounds.csv')
            kills.to_csv('../parsed-demos/csv/' + tournament + '_kills.csv')
            damages.to_csv('../parsed-demos/csv/' + tournament + '_damages.csv')
            grenades.to_csv('../parsed-demos/csv/' + tournament + '_grenades.csv')
            flashes.to_csv('../parsed-demos/csv/' + tournament + '_flashes.csv')
            weaponFires.to_csv('../parsed-demos/csv/' + tournament + '_weaponFires.csv')
            bombEvents.to_csv('../parsed-demos/csv/' + tournament + '_bombEvents.csv')
            frames.to_csv('../parsed-demos/csv/' + tournament + '_frames.csv')
            playerFrames.to_csv('../parsed-demos/csv/' + tournament + '_playerFrames.csv')

            f = open(tournament + '.txt', 'a')
            for demoName in temp_parsedDemos:
                f.write("%s\n" % demoName)
            f.close()

# Sandbox

In [6]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\antwerpmajor2022')

In [22]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\blastPremierWorldFinal2022')

../demo/blastPremierWorldFinal2022/faze-vs-g2-m1-nuke-2.dem done
../demo/blastPremierWorldFinal2022/faze-vs-g2-m1-nuke.dem done
../demo/blastPremierWorldFinal2022/faze-vs-g2-m2-inferno-2.dem done
../demo/blastPremierWorldFinal2022/faze-vs-g2-m2-inferno.dem done
../demo/blastPremierWorldFinal2022/faze-vs-g2-m3-ancient.dem done
../demo/blastPremierWorldFinal2022/g2-vs-vitality-m1-nuke.dem done
../demo/blastPremierWorldFinal2022/g2-vs-vitality-m2-vertigo.dem done
../demo/blastPremierWorldFinal2022/g2-vs-vitality-m3-anubis.dem done
../demo/blastPremierWorldFinal2022/heroic-vs-natus-vincere-m2-ancient.dem done
../demo/blastPremierWorldFinal2022/heroic-vs-natus-vincere-m3-nuke.dem done
../demo/blastPremierWorldFinal2022/heroic-vs-og-m1-inferno.dem done
../demo/blastPremierWorldFinal2022/heroic-vs-og-m2-mirage.dem done
../demo/blastPremierWorldFinal2022/heroic-vs-og-m3-anubis.dem done
../demo/blastPremierWorldFinal2022/liquid-vs-faze-m1-mirage.dem done
../demo/blastPremierWorldFinal2022/liqui

In [23]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\cologne2022')

../demo/cologne2022/astralis-vs-faze-m1-overpass.dem done
../demo/cologne2022/astralis-vs-faze-m2-inferno.dem done
../demo/cologne2022/cloud9-vs-astralis-m1-inferno.dem done
../demo/cologne2022/cloud9-vs-astralis-m2-mirage.dem done
../demo/cologne2022/cloud9-vs-outsiders-m1-dust2.dem done
../demo/cologne2022/cloud9-vs-outsiders-m2-ancient.dem done
../demo/cologne2022/ence-vs-vitality-m1-nuke.dem done
../demo/cologne2022/ence-vs-vitality-m2-dust2.dem done
../demo/cologne2022/ence-vs-vitality-m3-mirage.dem done
../demo/cologne2022/faze-vs-00nation-m1-ancient.dem done
../demo/cologne2022/faze-vs-00nation-m2-inferno.dem done
../demo/cologne2022/faze-vs-movistar-riders-m1-ancient.dem done
../demo/cologne2022/faze-vs-movistar-riders-m2-nuke.dem done
../demo/cologne2022/furia-vs-astralis-m1-nuke.dem done
../demo/cologne2022/furia-vs-astralis-m2-mirage.dem done
../demo/cologne2022/furia-vs-spirit-m1-nuke.dem done
../demo/cologne2022/furia-vs-spirit-m2-mirage.dem done
../demo/cologne2022/furia-

In [24]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\dallas2022')

../demo/dallas2022/astralis-vs-vitality-vertigo.dem done
../demo/dallas2022/big-vs-cloud9-m1-overpass.dem done
../demo/dallas2022/big-vs-cloud9-m2-dust2.dem done
../demo/dallas2022/big-vs-cloud9-m3-ancient.dem done
../demo/dallas2022/complexity-vs-nip-m1-vertigo.dem done
../demo/dallas2022/complexity-vs-nip-m2-overpass.dem done
../demo/dallas2022/ence-vs-cloud9-m1-mirage.dem done
../demo/dallas2022/ence-vs-cloud9-m2-overpass.dem done
../demo/dallas2022/ence-vs-cloud9-m3-ancient.dem done
../demo/dallas2022/ence-vs-faze-m1-mirage.dem done
../demo/dallas2022/ence-vs-faze-m2-ancient.dem done
../demo/dallas2022/ence-vs-faze-m3-nuke.dem done
../demo/dallas2022/ence-vs-furia-m1-vertigo.dem done
../demo/dallas2022/ence-vs-furia-m2-nuke.dem done
../demo/dallas2022/encore-vs-astralis-m1-vertigo.dem done
../demo/dallas2022/encore-vs-astralis-m2-nuke.dem done
../demo/dallas2022/faze-vs-cloud9-m1-overpass.dem done
../demo/dallas2022/faze-vs-cloud9-m2-inferno.dem done
../demo/dallas2022/faze-vs-clou

In [25]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\katowice2022')

../demo/katowice2022/astralis-vs-ence-m1-mirage.dem done
../demo/katowice2022/astralis-vs-ence-m2-overpass.dem done
../demo/katowice2022/astralis-vs-ence-m3-ancient.dem done
../demo/katowice2022/astralis-vs-fnatic-m1-inferno.dem done
../demo/katowice2022/astralis-vs-fnatic-m2-overpass.dem done
../demo/katowice2022/astralis-vs-fnatic-m3-ancient.dem done
../demo/katowice2022/astralis-vs-g2-m1-ancient.dem done
../demo/katowice2022/astralis-vs-g2-m2-dust2.dem done
../demo/katowice2022/astralis-vs-g2-m3-inferno.dem done
../demo/katowice2022/copenhagen-flames-vs-gambit-m1-overpass.dem done
../demo/katowice2022/copenhagen-flames-vs-gambit-m2-inferno.dem done
../demo/katowice2022/copenhagen-flames-vs-gambit-m3-vertigo.dem done
../demo/katowice2022/faze-vs-fnatic-m1-overpass.dem done
../demo/katowice2022/faze-vs-fnatic-m2-inferno.dem done
../demo/katowice2022/faze-vs-fnatic-m3-mirage.dem done
../demo/katowice2022/furia-vs-astralis-m1-overpass.dem done
../demo/katowice2022/furia-vs-astralis-m2-m

In [26]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\katowice2023')

../demo/katowice2023/big-vs-spirit-m1-nuke.dem done
../demo/katowice2023/big-vs-spirit-m2-mirage.dem done
../demo/katowice2023/big-vs-spirit-m3-anubis.dem done
../demo/katowice2023/cloud9-vs-ihc-m1-inferno.dem done
../demo/katowice2023/cloud9-vs-ihc-m2-mirage.dem done
../demo/katowice2023/cloud9-vs-natus-vincere-m1-ancient.dem done
../demo/katowice2023/cloud9-vs-natus-vincere-m2-inferno.dem done
../demo/katowice2023/complexity-vs-mouz-m1-inferno.dem done
../demo/katowice2023/complexity-vs-mouz-m2-nuke.dem done
../demo/katowice2023/complexity-vs-mouz-m3-ancient.dem done
../demo/katowice2023/faze-vs-ihc-m1-inferno.dem done
../demo/katowice2023/faze-vs-ihc-m2-ancient.dem done
../demo/katowice2023/faze-vs-liquid-m1-inferno.dem done
../demo/katowice2023/faze-vs-liquid-m2-overpass.dem done
../demo/katowice2023/faze-vs-liquid-m3-mirage.dem done
../demo/katowice2023/faze-vs-spirit-m1-ancient.dem done
../demo/katowice2023/faze-vs-spirit-m2-nuke.dem done
../demo/katowice2023/fnatic-vs-complexity

In [27]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\proleague16')

../demo/proleague16/astralis-vs-heet-m1-dust2.dem done
../demo/proleague16/astralis-vs-heet-m2-inferno.dem done
../demo/proleague16/astralis-vs-heet-m3-nuke.dem done
../demo/proleague16/astralis-vs-mouz-m1-inferno.dem done
../demo/proleague16/astralis-vs-mouz-m2-nuke.dem done
../demo/proleague16/big-vs-ftw-m1-inferno.dem done
../demo/proleague16/big-vs-ftw-m2-dust2.dem done
../demo/proleague16/big-vs-ftw-m3-nuke.dem done
../demo/proleague16/big-vs-mibr-m1-vertigo.dem done
../demo/proleague16/big-vs-mibr-m2-nuke.dem done
../demo/proleague16/big-vs-mibr-m3-dust2.dem done
../demo/proleague16/cloud9-vs-eternal-fire-m1-dust2.dem done
../demo/proleague16/cloud9-vs-eternal-fire-m2-overpass.dem done
../demo/proleague16/cloud9-vs-evil-geniuses-m1-inferno.dem done
../demo/proleague16/cloud9-vs-evil-geniuses-m2-dust2.dem done
../demo/proleague16/cloud9-vs-faze-m1-inferno.dem done
../demo/proleague16/cloud9-vs-faze-m2-ancient.dem done
../demo/proleague16/cloud9-vs-faze-m3-mirage.dem done
../demo/p

In [28]:
parse_demos_in_folder(r'D:\bme\MSc\csgo\data-scape\awpy\scrape-parse\demo\riomajor2022')

../demo/riomajor2022/big-vs-fnatic-m1-overpass.dem done
../demo/riomajor2022/big-vs-fnatic-m2-mirage.dem done
../demo/riomajor2022/big-vs-natus-vincere-m1-mirage.dem done
../demo/riomajor2022/big-vs-natus-vincere-m2-dust2.dem done
../demo/riomajor2022/ence-vs-bad-news-eagles-ancient.dem done
../demo/riomajor2022/ence-vs-fnatic-nuke.dem done
../demo/riomajor2022/ence-vs-vitality-m1-vertigo.dem done
../demo/riomajor2022/ence-vs-vitality-m2-nuke.dem done
../demo/riomajor2022/faze-vs-bad-news-eagles-m1-vertigo.dem done
../demo/riomajor2022/faze-vs-bad-news-eagles-m2-nuke.dem done
../demo/riomajor2022/faze-vs-bad-news-eagles-m3-mirage.dem done
../demo/riomajor2022/faze-vs-cloud9-overpass.dem done
../demo/riomajor2022/faze-vs-vitality-inferno.dem done
../demo/riomajor2022/furia-vs-big-m1-nuke.dem done
../demo/riomajor2022/furia-vs-big-m2-vertigo.dem done
../demo/riomajor2022/furia-vs-ence-vertigo.dem done
../demo/riomajor2022/furia-vs-natus-vincere-m1-nuke.dem done
../demo/riomajor2022/furia